In [1]:
import sys

sys.path.append('/dss/dsshome1/lxc0C/ge32luv/myenv/lib/python3.9/site-packages/')
sys.path.reverse()

import site
import importlib
import pkg_resources

site.addsitedir('/dss/dsshome1/lxc0C/ge32luv/myenv/lib/python3.9/site-packages/')

importlib.reload(pkg_resources)
pkg_resources.get_distribution('google-api-core')

import tensorflow as tf
import numpy as np
import pandas as pd
import random
import os
import struct
from array import array
from os.path  import join
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageFilter
import scipy.io

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from datetime import datetime
import yaml
from pathlib import Path
from sklearn import manifold
import pickle 

import seaborn as sns
import saliency.core as saliency

import warnings

warnings.filterwarnings("ignore")
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)


2026-01-29 10:54:32.339352: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib64/openmpi/lib:/usr/local/cuda/lib64:/opt/intel/oneapi/redist/lib:/opt/postgresql/15.2/lib
2026-01-29 10:54:32.339380: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import model
from dataLoader import DataLoader

2026-01-29 10:54:39.046661: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2026-01-29 10:54:39.046699: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (qlm): /proc/driver/nvidia/version does not exist
2026-01-29 10:54:39.047198: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
trained_mltqnn_models = {'sat': {'0.1': 'doublefine/Generalizability_TNNLS/sat_16_QCNN_patch_U3locWeight_weightRatio5.0_234244', 
                               '0.5': 'doublefine/Generalizability_TNNLS/sat_18_QCNN_patch_U3locWeight_weightRatio5.0_184406', 
                               '1':'doublefine/Generalizability_TNNLS/sat_17_QCNN_patch_U3locWeight_weightRatio5.0_133305'},
                       'lcz': {'0.1': 'doublefine/Generalizability_TNNLS/lcz_22_QCNN_patch_U3locWeight_weightRatio5.0_191037', 
                               '0.5': 'doublefine/Generalizability_TNNLS/lcz_19_QCNN_patch_U3locWeight_weightRatio5.0_161422', 
                               '1':'doublefine/Generalizability_TNNLS/lcz_17_QCNN_patch_U3locWeight_weightRatio5.0_190015'},
                       'eurosat': {'0.1': 'doublefine/Generalizability_TNNLS/eurosat_16_QCNN_patch_U3locWeight_weightRatio3.0_070843', 
                                   '0.5': 'doublefine/Generalizability_TNNLS/eurosat_18_QCNN_patch_U3locWeight_weightRatio10.0_174226', 
                                   '1':'doublefine/Generalizability_TNNLS/eurosat_16_QCNN_patch_U3locWeight_weightRatio5.0_030642'},  
                       'patternet': {'0.1': 'doublefine/Generalizability_TNNLS/patternet_16_QCNN_patch_U3locWeight_weightRatio10.0_092139', 
                                     '0.5': 'doublefine/Generalizability_TNNLS/patternet_17_QCNN_patch_U3locWeight_weightRatio10.0_005504', 
                                     '1':'doublefine/Generalizability_TNNLS/patternet_16_QCNN_patch_U3locWeight_weightRatio7.0_074300'},
                       'location': 'experiments/wandb/'}

saved_models = {'qcnn': trained_mltqnn_models}



In [4]:
def get_model(saved_models, target_model, dataset, label_ratio):
    target_run = saved_models[target_model][dataset][str(label_ratio)]
    wandb_path = saved_models[target_model]['location']
    qcnn_model, test_acc = model.build_model(target_model, dataset, label_ratio, labeled_train_x.shape[1:], len(class_name), target_run, wandb_path)
    test = model.matrices(qcnn_model, test_x, test_y, 'test')
    assert test_acc == test[3]
    
    return qcnn_model

label_ratios = [0.1, 0.5, 1]
datasets = ['sat', 'lcz', 'eurosat', 'patternet']
model_type = 'qcnn'


for dataset in datasets:
    for label_ratio in label_ratios:
        print(f'dataset: {dataset}, label_ratio: {label_ratio}')
        data = DataLoader(dataset, label_ratio)
        unlabeled_train_x, unlabeled_train_y, labeled_train_x, labeled_train_y, valid_x, valid_y, test_x, test_y = data.get_data()
        class_name = data.get_categories()
        obtained_model = get_model(saved_models, model_type, dataset, label_ratio)

dataset: sat, label_ratio: 0.1
38/38 [==============================] - 36s 809ms/step - loss: 0.2269 - classifier_loss: 0.2189 - reconstruction_loss: 0.0080 - classifier_accuracy: 0.9150
dataset: sat, label_ratio: 0.5
38/38 [==============================] - 32s 708ms/step - loss: 0.1526 - classifier_loss: 0.1451 - reconstruction_loss: 0.0075 - classifier_accuracy: 0.9525
dataset: sat, label_ratio: 1
38/38 [==============================] - 34s 753ms/step - loss: 0.1109 - classifier_loss: 0.1045 - reconstruction_loss: 0.0064 - classifier_accuracy: 0.9592
dataset: lcz, label_ratio: 0.1
63/63 [==============================] - 52s 736ms/step - loss: 0.2668 - classifier_loss: 0.2606 - reconstruction_loss: 0.0062 - classifier_accuracy: 0.9080
dataset: lcz, label_ratio: 0.5
63/63 [==============================] - 52s 757ms/step - loss: 0.1964 - classifier_loss: 0.1909 - reconstruction_loss: 0.0055 - classifier_accuracy: 0.9335
dataset: lcz, label_ratio: 1
63/63 [==========================